In [47]:
import gzip
import pyBigWig
import tqdm

In [52]:
chrom_sizes = []
with open("/Users/weisburd/p1/ref/GRCh38/hg38.fa.fai") as f:
    for line in f:
        fields = line.split("\t")
        chrom = fields[0]
        size = int(fields[1])
        if "_" in chrom or "-" in chrom or "ebv" in chrom.lower():
            continue
        chrom_sizes.append((chrom, size))

[('chr1', 248956422),
 ('chr2', 242193529),
 ('chr3', 198295559),
 ('chr4', 190214555),
 ('chr5', 181538259),
 ('chr6', 170805979),
 ('chr7', 159345973),
 ('chr8', 145138636),
 ('chr9', 138394717),
 ('chr10', 133797422),
 ('chr11', 135086622),
 ('chr12', 133275309),
 ('chr13', 114364328),
 ('chr14', 107043718),
 ('chr15', 101991189),
 ('chr16', 90338345),
 ('chr17', 83257441),
 ('chr18', 80373285),
 ('chr19', 58617616),
 ('chr20', 64444167),
 ('chr21', 46709983),
 ('chr22', 50818468),
 ('chrX', 156040895),
 ('chrY', 57227415),
 ('chrM', 16569)]

In [65]:
info_splice_ai_fields = "ALLELE|SYMBOL|DS_AG|DS_AL|DS_DG|DS_DL|DP_AG|DP_AL|DP_DG|DP_DL".split("|")

bins = {
    0: 0,
    0.2: 0,
    0.3: 0,
    0.5: 0,
    0.8: 0,
}

bw = pyBigWig.open("spliceAI.snv.hg38.all.bigWig", "w")
bw.addHeader(chrom_sizes)         

#total_num_lines = 3_433_386_166
total_num_lines = 280_000_000
with gzip.open("./spliceai_scores.raw.snv.hg38.vcf.gz", "rt") as f:
    line = ''
    while not line.startswith("#CHROM"):
        line = next(f)

    prev_chrom = None
    prev_pos = None
    current_scores = []
    for i, line in enumerate(tqdm.notebook.tqdm(f, unit=" lines", total=total_num_lines)):
        line = line.rstrip()
        fields = line.split("\t")
        chrom = fields[0]
        pos = int(fields[1])
        if pos != prev_pos and current_scores:
            max_score = max(current_scores)
            if max_score >= 0.2:
                #bw.write(prev_chrom, prev_pos - 1, prev_pos, max_score)
                bw.addEntries(f"chr{prev_chrom}", [prev_pos], values=[max_score], span=1)
            current_scores.clear()
            prev_chrom = chrom
            prev_pos = pos
            
        #ref = fields[3]
        #alt = fields[4]
        idx = fields[7].index("=") + 1
        delta_scores = dict(zip(info_splice_ai_fields, fields[7][idx:].split("|")))
        max_delta = max(float(delta_scores[k]) for k in ('DS_AG', 'DS_AL', 'DS_DG', 'DS_DL'))
        current_scores.append(max_delta)
        for k in bins:
            if max_delta >= k:
                bins[k] += 1
        
        #if i > 100_000_0000:
        if i > total_num_lines:
            break
bw.close()

for k, v in bins.items():
    print(f"{v:10d} rows >= {k:0.2f} {(100.0*v)/bins[0]:3.1f}%")


 280000002 rows >= 0.00 100.0%
   1887202 rows >= 0.20 0.7%
   1152851 rows >= 0.30 0.4%
    625737 rows >= 0.50 0.2%
    328888 rows >= 0.80 0.1%


In [61]:
140000*10*60

84000000

In [43]:
x = [1,2]

In [45]:
x.clear()

In [46]:
x

[]